In [22]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly

import test
import mylayers
import util

# Used to save temporary variables
import pickle

## Read data

In [23]:
df_train = pd.read_csv("./data/mnist_train.csv", header=None)
df_valid = pd.read_csv("./data/mnist_test.csv", header=None)


### Shuffle the data set

In [24]:
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)


### Seperate feature column and target columns

In [25]:
X_train = np.array(df_train.drop(0, axis=1), dtype="f")
Y_train = df_train.loc[:, 0]
X_valid = np.array(df_valid.drop(0, axis=1), dtype="f")
Y_valid = df_valid.loc[:, 0]


### One hot encoding Y

In [26]:
encoder = util.onehot_encoder(Y_train)
Y_train = encoder.encoding(Y_train)
Y_valid = encoder.encoding(Y_valid)


In [27]:
# Testing
# X_train = X_train[:1000]
# Y_train = Y_train[:1000]


## Pretrain

### First FC

In [28]:
X_train_z_scored = mylayers.InputLayer(X_train).forward(X_train)
layers_pre_1 = []
layers_pre_1.append(mylayers.InputLayer(X_train))
# ---In---
fc_layer = mylayers.FullyConnectedLayer(X_train.shape[1], 128)
layers_pre_1.append(fc_layer)
# ---Out---
fc_layer = mylayers.FullyConnectedLayer(128, X_train.shape[1])
layers_pre_1.append(fc_layer)
# ---valid---
layers_pre_1.append(mylayers.SquaredError())

In [29]:
%time
hyperparams = {'epoch': 100, 'eta': 10e-4, 'batch_size':1000, 'rho1': 0.9, 'rho2': 0.999}
epoch = hyperparams['epoch']
eta = hyperparams['eta']
batch_size = hyperparams['batch_size']
rho1 = hyperparams['rho1']
rho2 = hyperparams['rho2']
ls_scores_pre_1,Y_h,_= test.adam_learn(X_train,X_train_z_scored,layers_pre_1,mylayers.SquaredError,epoch=epoch,eta=eta,print_eval=False,print_layer=False,batch_size=batch_size)


Wall time: 0 ns


100%|██████████| 100/100 [09:48<00:00,  5.89s/it]


In [30]:
util.draw_scores(ls_scores_pre_1,"None",hyperparams,"SE Score")

In [31]:
h_1 = layers_pre_1[1].getPrevOut()

### Second FC

In [32]:
layers_pre_2 = []
layers_pre_2.append(mylayers.InputLayer(h_1,z_score=False))
# ---In---
fc_layer = mylayers.FullyConnectedLayer(128, 64)
layers_pre_2.append(fc_layer)
# ---Out---
fc_layer = mylayers.FullyConnectedLayer(64, 128)
layers_pre_2.append(fc_layer)
# ---valid---
layers_pre_2.append(mylayers.SquaredError())

In [33]:
%time
hyperparams = {'epoch': 1000, 'eta': 10e-5, 'batch_size':1000, 'rho1': 0.9, 'rho2': 0.999}
epoch = hyperparams['epoch']
eta = hyperparams['eta']
batch_size = hyperparams['batch_size']
rho1 = hyperparams['rho1']
rho2 = hyperparams['rho2']
ls_scores_train,Y_hat,acc= test.adam_learn(h_1,h_1,layers_pre_2,mylayers.SquaredError,epoch=epoch,eta=eta,print_eval=False,print_layer=False,batch_size=batch_size)


Wall time: 0 ns


100%|██████████| 1000/1000 [00:09<00:00, 102.57it/s]


In [34]:
util.draw_scores(ls_scores_train,"None",hyperparams,"SE Score")

In [35]:
h_2 = layers_pre_2[1].getPrevOut()

### Third FC

In [36]:
layers_pre_3 = []
layers_pre_3.append(mylayers.InputLayer(h_2,z_score=False))
# ---In---
fc_layer = mylayers.FullyConnectedLayer(64, 10)
layers_pre_3.append(fc_layer)
# ---Out---
fc_layer = mylayers.FullyConnectedLayer(10, 64)
layers_pre_3.append(fc_layer)
# ---valid---
layers_pre_3.append(mylayers.SquaredError())

In [37]:
%time
hyperparams = {'epoch': 10000, 'eta': 10e-6, 'batch_size':1000, 'rho1': 0.9, 'rho2': 0.999}
epoch = hyperparams['epoch']
eta = hyperparams['eta']
batch_size = hyperparams['batch_size']
rho1 = hyperparams['rho1']
rho2 = hyperparams['rho2']
ls_scores_train,Y_hat,acc= test.adam_learn(h_2,h_2,layers_pre_3,mylayers.SquaredError,epoch=epoch,eta=eta,print_eval=False,print_layer=False,batch_size=batch_size)


Wall time: 0 ns


100%|██████████| 10000/10000 [00:44<00:00, 224.27it/s]


In [38]:
util.draw_scores(ls_scores_train,"None",hyperparams,"SE Score")

In [39]:
h_3 = layers_pre_3[1].getPrevOut()

## Build layers

In [40]:
layers = []
# ---Input Layer---
layers.append(mylayers.InputLayer(X_train))
# ---Hidden Layer---
# Greedy pre trained
fc_layer = layers_pre_1[1]
layers.append(fc_layer)
layers.append(mylayers.ReLuLayer())
# Greedy pre trained
fc_layer = layers_pre_2[1]
layers.append(fc_layer)
layers.append(mylayers.ReLuLayer())
# Greedy pre trained
fc_layer = layers_pre_3[1]
layers.append(fc_layer)
layers.append(mylayers.SoftmaxLayer())
# ---Objective---
layers.append(mylayers.CrossEntropy())


In [41]:
layers

### layers test

In [42]:
%time
hyperparams = {'epoch': 500, 'eta': 10e-6, 'batch_size': 100, 'rho1': 0.9, 'rho2': 0.999}
epoch = hyperparams['epoch']
eta = hyperparams['eta']
batch_size = hyperparams['batch_size']
rho1 = hyperparams['rho1']
rho2 = hyperparams['rho2']
ls_scores_train,Y_hat,ls_accuracy_train,ls_scores_valid,ls_accuracy_valid = test.adam_learn_with_validation(X_train,Y_train,X_valid,Y_valid,layers,mylayers.CrossEntropy,epoch=epoch,eta=eta,print_eval=False,print_layer=False,batch_size=batch_size)


Wall time: 0 ns


100%|██████████| 500/500 [34:00<00:00,  4.08s/it]


In [43]:
train_results = {
    'ls_scores_train':ls_scores_train,
    'Y_hat':Y_hat,
    'ls_accuracy_train':ls_accuracy_train
    ,'ls_scores_valid':ls_scores_valid,
    'ls_accuracy_valid':ls_accuracy_valid
}

In [44]:
# save result to pickle
with open('models/Greedy_MLP_train_results_relu_relu_softmax.pkl', 'wb') as f:
    pickle.dump(train_results, f)

In [45]:
# read result from pickle
with open('models/Greedy_MLP_train_results_relu_relu_softmax.pkl', 'rb') as f:
    train_results = pickle.load(f)
ls_scores_train = train_results['ls_scores_train']
Y_hat = train_results['Y_hat']
ls_accuracy_train = train_results['ls_accuracy_train']
ls_scores_valid = train_results['ls_scores_valid']
ls_accuracy_valid = train_results['ls_accuracy_valid']

In [46]:
# graph of scores
hidden_layers = "relu -> relu -> softmax"
fig = util.draw_scores_with_validation(
    ls_scores_train, ls_scores_valid, hidden_layers, hyperparams
)
fig.show()


In [47]:
# graph of accuracys
fig = util.draw_accuracys_with_validation(
    ls_accuracy_train, ls_accuracy_valid, hidden_layers, hyperparams
)
fig.show()


### validate accuracy

In [48]:
labeled_Y_hat = encoder.decoding(Y_hat)
labeled_Y_hat


[7,
 3,
 8,
 9,
 3,
 9,
 7,
 7,
 5,
 4,
 2,
 5,
 6,
 8,
 1,
 7,
 4,
 8,
 0,
 7,
 9,
 3,
 1,
 7,
 6,
 5,
 3,
 3,
 3,
 6,
 0,
 8,
 4,
 9,
 5,
 1,
 8,
 7,
 8,
 8,
 0,
 8,
 2,
 0,
 8,
 5,
 5,
 7,
 2,
 0,
 1,
 5,
 9,
 3,
 0,
 6,
 2,
 2,
 6,
 4,
 0,
 1,
 8,
 4,
 0,
 2,
 4,
 7,
 7,
 4,
 4,
 0,
 1,
 4,
 8,
 6,
 0,
 0,
 0,
 9,
 6,
 3,
 6,
 9,
 9,
 4,
 8,
 6,
 6,
 1,
 9,
 5,
 2,
 6,
 3,
 4,
 9,
 1,
 0,
 4,
 2,
 8,
 5,
 4,
 4,
 4,
 4,
 8,
 2,
 4,
 5,
 4,
 3,
 1,
 3,
 5,
 4,
 2,
 1,
 1,
 2,
 2,
 8,
 2,
 7,
 0,
 4,
 0,
 6,
 3,
 5,
 2,
 2,
 5,
 1,
 1,
 4,
 5,
 2,
 2,
 9,
 3,
 6,
 2,
 9,
 9,
 5,
 3,
 4,
 8,
 7,
 3,
 9,
 7,
 9,
 9,
 0,
 1,
 8,
 0,
 5,
 6,
 0,
 2,
 8,
 0,
 6,
 7,
 1,
 8,
 3,
 8,
 2,
 9,
 4,
 5,
 0,
 1,
 2,
 1,
 3,
 1,
 7,
 8,
 8,
 1,
 8,
 8,
 9,
 9,
 8,
 8,
 0,
 1,
 5,
 7,
 4,
 3,
 7,
 4,
 4,
 4,
 1,
 9,
 2,
 2,
 1,
 5,
 0,
 4,
 6,
 5,
 1,
 6,
 6,
 8,
 3,
 2,
 2,
 5,
 2,
 8,
 7,
 6,
 0,
 2,
 9,
 9,
 0,
 5,
 3,
 4,
 3,
 3,
 6,
 7,
 2,
 7,
 0,
 5,
 9,
 6,
 8,
 0,
 1,
 5,
 9,
 0,
 6,
 9,


In [49]:
labeled_Y = encoder.decoding(Y_train)
labeled_Y


[7,
 3,
 8,
 9,
 3,
 9,
 7,
 7,
 5,
 4,
 2,
 5,
 6,
 8,
 1,
 7,
 4,
 8,
 0,
 7,
 9,
 3,
 1,
 7,
 6,
 5,
 3,
 3,
 3,
 6,
 0,
 8,
 4,
 9,
 5,
 1,
 8,
 7,
 8,
 8,
 0,
 8,
 2,
 0,
 8,
 5,
 5,
 2,
 2,
 0,
 1,
 5,
 9,
 3,
 0,
 6,
 2,
 2,
 6,
 4,
 0,
 1,
 8,
 4,
 0,
 2,
 4,
 7,
 7,
 4,
 4,
 0,
 1,
 4,
 8,
 6,
 0,
 0,
 0,
 9,
 6,
 3,
 6,
 9,
 9,
 4,
 8,
 6,
 6,
 1,
 9,
 5,
 2,
 6,
 3,
 4,
 9,
 1,
 0,
 4,
 2,
 8,
 5,
 4,
 4,
 4,
 4,
 8,
 2,
 4,
 5,
 4,
 3,
 1,
 3,
 5,
 4,
 2,
 1,
 1,
 2,
 2,
 8,
 2,
 7,
 0,
 4,
 0,
 6,
 3,
 5,
 2,
 2,
 5,
 1,
 1,
 4,
 5,
 2,
 2,
 9,
 3,
 6,
 2,
 9,
 9,
 5,
 3,
 4,
 8,
 7,
 3,
 9,
 7,
 9,
 9,
 0,
 1,
 8,
 0,
 5,
 6,
 0,
 2,
 8,
 0,
 6,
 7,
 1,
 8,
 3,
 8,
 2,
 9,
 4,
 5,
 0,
 1,
 2,
 1,
 3,
 1,
 7,
 8,
 8,
 1,
 8,
 8,
 9,
 9,
 8,
 8,
 0,
 1,
 5,
 7,
 4,
 3,
 7,
 4,
 4,
 4,
 1,
 9,
 2,
 2,
 1,
 5,
 0,
 4,
 6,
 5,
 1,
 6,
 6,
 8,
 3,
 2,
 2,
 5,
 2,
 8,
 7,
 6,
 0,
 2,
 9,
 9,
 0,
 5,
 3,
 4,
 3,
 3,
 6,
 7,
 2,
 7,
 0,
 5,
 9,
 6,
 8,
 0,
 1,
 5,
 9,
 0,
 6,
 9,


In [50]:
c = 0
for i in range(len(labeled_Y)):
    if labeled_Y[i] == labeled_Y_hat[i]:
        c += 1
print("Final Accuracy:", (c / len(labeled_Y)) * 100, "%")


Final Accuracy: 99.68166666666667 %
